## Vesuvius Challenge - Ink Detection Training Notebook

## Note

- 高い解像度でリサイズすることはprecisionの向上につながるため有効
- seresnextでチャネル間の相関を見れるので有効
- 文字の太さ・書き方に大きくバリアントがあるので、横の相関よりもdepthの相関の方が大事かもしれない
- 深さに関して、隣り合う深さ同士に大きな変化はない
- fpをfnよりも小さくしたい
- valid_scoreはCFG.thd依存
- valid_lossはCFG.loss1/loss2依存
- encoder内でdepthをクロップしてバッチで繋げた方が精度が良い
- maskに対しては有効ではないが、labelは的確なラベルを用いることで精度が向上
- depthは22~34 or 24 ~ 36
- 画像のサイズは大きい方が良いのか？（BUFFER / SHARED_HEIGHT）
    - 比率を同じにして試してみる
    - SHARED_HEIGHTをデカくするとデータがメモリに載らなさそうだった
- NetのweightsにBatchNormはない方が良い
- BUFFER:strideを160:96から160:80にしたら精度が落ちた
    - trainの精度は上がっていた
    - BUFFERに対してstrideが細かすぎると過学習に繋がっているのかもしれない
- 文字が見えなくなるよりも、高いthdを設定して、文字が大きく見えた方が良い（仮説）
- 学習時にはstrideは大きくし、過学習を防ぐ。識別時にはstrideは小さくし見落としを減らす
- バイリニアよりもバイキュービックが良い
- ノイズについて
    - 大きすぎると良くない(intensityが(0.0001, 0.0005)くらいが良かった)
        - 元画像が255 * 255の範囲であることを踏まえると、1/255の変化で255のずれがある
- lossのαとβについて
    - 理論的にはαは大きい方が良い（偽陽性fpはスコアを大きく下げるので）
    - しかし、thdを用いて偽陽性を防ぐ手法もあるため、学習時にはあえて偽陰性に強いペナルティを与えた方が良いのかもしれない
    - Stacked UnetのI層目では高いβで見落としをなくして、二層目で高いαにしてノイズを除去する作戦
- 文字サイズとリサイズサイズについて
- thresholdを変えるだけでpublic lbが大きく変わる
    - 0.5から0.7にしたら0.05上がった（0.58→0.63）
- 学習時・推論時に低いRESIZED_HEIGHTで進めると、提出時にcvとlbに差が出るかもしれない。これは最終的にリサイズする際に細かい部分の見落としが発生するからだと思う
    - 良い学習をするために、正しいラベル・正しい入力が適切→resizeすることでデータの質が損なわれているのかも
- [ ] 3dcnnにbatch化したdepthを横に繋げる実装が使えるか考える
- [ ] Stacked Unetを3dcnnに適用してみる
- 解像度（BUFFER）はある程度大きくないといけない（<112）
- DecoderBlockのattention=scseについて
- 3dresnetで見る場合、最初のconv1とmaxpoolで解像度が1/4に縮小されるので、元が128x128であれば32x32をUpScaleすることになる。なので、元々ある程度高い解像度が必要
- 3dresnetで見る場合、Z_LISTは2飛ばしで広く見る方が精度が高い
- ampを有効にする（precision=16）
- 3dresnetで学習時の精度が良くならない
    - 解像度が低い（仮説）
    - agumentationが難しすぎる（仮説）
        - augmentationの確率を下げても変化はなかった
        - augmentationをなくすとどうなる？

### Setup

In [1]:
# # Pretrained weights
# # ref - https://github.com/kenshohara/3D-ResNets-PyTorch
# !pip install gdown
# !gdown 1Nb4abvIkkp_ydPFA9sNPT1WakoVKA8Fa

# # Utility packages for reading and visualizing volumes
# !pip install zarr imageio-ffmpeg

# # save model checkpoints
# !mkdir ./ckpts

# !kaggle datasets download -d samfc10/vesuvius-zarr-files

In [2]:

import numpy as np
import torch
import torch.nn as nn
import wandb
import torchvision
import datetime
import imageio
# import cupy
from sklearn.model_selection import KFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning
import segmentation_models_pytorch as smp
import pytorch_lightning as pl
import pytorch_lightning.callbacks.model_checkpoint
import pytorch_lightning.plugins
from skimage.transform import resize as resize_ski
from pytorch_lightning.strategies.ddp import DDPStrategy
from pytorch_lightning.loggers import WandbLogger
from einops import rearrange, reduce, repeat
import torch.nn.functional as F
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder, DecoderBlock
from timm.models.resnet import resnet10t, resnet34d, resnet50d, resnet14t, seresnext26d_32x4d, seresnext50_32x4d
import os
import torch.utils.data
from dataclasses import dataclass

from scipy.ndimage import distance_transform_edt

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import os,cv2
import gc
import sys
import matplotlib.patches as patches
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import pytorch_lightning as pl

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
import albumentations as A
from IPython.display import Video

# sys.path.append("/home/fummicc1/codes/competitions/kaggle-ink-detection")
# sys.path.append("/kaggle/input/resnet3d")
from resnet3d import generate_model

pytorch_lightning.seed_everything(seed=42)
torch.set_float32_matmul_precision('high')


@dataclass
class CFG():
    # Data config
    # DATA_DIR = '/kaggle/input/vesuvius-challenge-ink-detection/'
    DATA_DIR = '/home/fummicc1/codes/competitions/kaggle-ink-detection'
    # DATA_DIR = '/home/fummicc1/codes/Kaggle/kaggle-ink-detection'
    BUFFER = 160 # Half-size of papyrus patches we'll use as model inputs
    CROP_SIZE = BUFFER * 2
    STRIDE = 96
    # Z_LIST = list(range(0, 20, 5)) + list(range(22, 34))  # Offset of slices in the z direction
    # Z_LIST = [20, 22, 24, 26] + list(range(28, 36)) + [36, 38]
    # Z_LIST = list(range(16, 48, 2))
    Z_LIST = list(range(20, 40, 2))
    # Z_LIST = list(range(0, 24, 8)) + list(range(24, 36, 2)) + list(range(36, 64, 10))
    Z_DIM = len(Z_LIST)  # Number of slices in the z direction. Max value is 64 - Z_START
    # BATCH_Z_DIFF = 4
    SHARED_HEIGHT = 5120  # Max height to resize all papyrii

    # Model config
    BATCH_SIZE = 96

    device = torch.device("cuda")
    threshold = 0.5
    num_workers = 8
    exp = 1e-7
    mask_padding = BUFFER

    num_epochs = 20
    lr = 1e-3
    eta_min_lr = 5e-5
    WANDB_NOTE = "augmentationをさらに減らした"
    loss1_alpha = 0.5
    loss1_beta = 0.5
    loss2_alpha = 0.5
    loss2_beta = 0.5
    
    loss1_weight = 0.5
    loss2_weight = 0.5
    
    lr_scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts
    loss1 = smp.losses.TverskyLoss(
        smp.losses.BINARY_MODE,
        log_loss=False,
        from_logits=True, 
        smooth=1e-7,
        alpha=loss1_alpha,
        beta=loss1_beta,
    )
    loss2 = smp.losses.TverskyLoss(
        smp.losses.BINARY_MODE,
        log_loss=False,
        from_logits=True, 
        smooth=1e-7,
        alpha=loss2_alpha,
        beta=loss2_beta,
    )
    noise_intensity = (0.0001, 0.0005)
    use_new_label_mask = True
    pretrained = True
    MODEL_DEPTH = 34
    
    exp_name = "006-gpu17"
    prev_exp_name = "006-gpu17"
    
def class2dict(c):
    return {attr: getattr(c, attr) for attr in dir(c) if not callable(getattr(c, attr)) and not attr.startswith("__")}

### Config

### Load data

In [ ]:
def resize(img):
    current_height, current_width = img.shape    
    aspect_ratio = current_width / current_height
    if CFG.SHARED_HEIGHT is None:
        return img
    # new_height = CFG.SHARED_HEIGHT
    # pad_y = new_height - current_height
    # if pad_y > 0:
    #     # 元画像が小さい場合は解像度を大きくしないでpaddingをつける
    #     img = np.pad(img, [(0, pad_y), (0, 0)], constant_values=0)
    # else:
    # 既に十分でかい場合はリサイズする
    # 本当はpaddingしたいけど、メモリサイズが大きくなる
    new_height = CFG.SHARED_HEIGHT
    new_width = int(CFG.SHARED_HEIGHT * aspect_ratio)
    new_size = (new_width, new_height)
    # (W, H)の順で渡すが結果は(H, W)になっている
    img = cv2.resize(img, new_size)
    return img

def load_mask(split, index): 
    if index == "2a" or index == "2b":
        mode = index[1]
        index = "2"
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/mask.png", 0) // 255
    if index == "2":
        h = 9456
        if mode == "a":
            img = img[h:, :]
        elif mode == "b":   
            img = img[:h, :]
    img = resize(img)
    img = np.pad(img, 1, constant_values=0)
    dist = distance_transform_edt(img)
    img[dist <= CFG.mask_padding] = 0
    img = img[1:-1, 1:-1]    
    return img


def load_labels(split, index):
    if index == "2a" or index == "2b":
        mode = index[1]
        index = "2"
    suffix = "_new" if CFG.use_new_label_mask else ""
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/inklabels{suffix}.png", 0) // 255    
    if index == "2":
        h = 9456
        if mode == "a":
            img = img[h:, :]
        elif mode == "b":   
            img = img[:h, :]
    img = resize(img)
    return img




def load_ir(split, index):
    if index == "2a" or index == "2b":
        mode = index[1]
        index = "2"
    img = cv2.imread(f"{CFG.DATA_DIR}/{split}/{index}/ir.png", 0)
    if index == "2":
        h = 9456
        if mode == "a":
            img = img[h:, :]
        elif mode == "b":   
            img = img[:h, :]
    img = resize(img)
    return img

In [ ]:
def load_volume(split, index):
    if index == "2a" or index == "2b":
        mode = index[1]
        index = "2"
    # Load the 3d x-ray scan, one slice at a time
    all = sorted(glob.glob(f"{CFG.DATA_DIR}/{split}/{index}/surface_volume/*.tif"))
    z_slices_fnames = [all[i] for i in range(len(all)) if i in CFG.Z_LIST]
    assert len(z_slices_fnames) == CFG.Z_DIM
    z_slices = []
    for z, filename in  tqdm(enumerate(z_slices_fnames)):
        img = cv2.imread(filename, -1)
        if index == "2":
            h = 9456
            if mode == "a":
                img = img[h:, :]
            elif mode == "b":
                img = img[:h, :]
        img = resize(img)
        # img = (img / (2 ** 8)).astype(np.uint8)
        img = img.astype(np.float32) // 255
        z_slices.append(img)
    return np.stack(z_slices, axis=-1)

In [ ]:
def is_in_masked_zone(location, mask):
    return mask[location[0], location[1]] > 0

In [ ]:
def generate_locations_ds(volume, mask, label=None, skip_zero=False):
    is_in_mask_train = lambda x: is_in_masked_zone(x, mask)

    # Create a list to store train locations
    locations = []

    # Generate train locations
    volume_height, volume_width = volume.shape[:-1]

    for y in range(CFG.BUFFER, volume_height - CFG.BUFFER, CFG.STRIDE):
        for x in range(CFG.BUFFER, volume_width - CFG.BUFFER, CFG.STRIDE):
            if skip_zero and label is not None and np.all(label[y - CFG.BUFFER // 2 : y + CFG.BUFFER // 2, x - CFG.BUFFER // 2 : x + CFG.BUFFER // 2] == 0):
                # print(f"skip location at (y: {y}, x: {x})")
                continue
            if is_in_mask_train((y, x)):
                locations.append((y, x))

    # Convert the list of train locations to a PyTorch tensor
    locations_ds = np.stack(locations, axis=0)
    return locations_ds

In [ ]:
FRAGMENTS_ZARR = {
    "1" : {},
    "2a" : {},
    "2b" : {},
    "3" : {},
}


In [ ]:
for key in ["1", "2a", "2b", "3"]:
    FRAGMENTS_ZARR[key]["surface_volume"] = load_volume("train", key)
    FRAGMENTS_ZARR[key]["mask"] = load_mask("train", key)
    FRAGMENTS_ZARR[key]["truth"] = load_labels("train", key)
    FRAGMENTS_ZARR[key]["infrared"] = load_ir("train", key)

In [ ]:
FRAGMENTS_SHAPE = {k : v["surface_volume"].shape[:-1] for k, v in FRAGMENTS_ZARR.items()}

In [ ]:
list(FRAGMENTS_ZARR["1"].keys())
# np.unique(np.array(FRAGMENTS_ZARR["1"]["surface_volume"]))

In [ ]:
plt.imshow(FRAGMENTS_ZARR["2a"]["truth"])

In [ ]:
plt.imshow(FRAGMENTS_ZARR["1"]["truth"][:,:])

### Visualise input

In [ ]:
fragment_id = FRAGMENTS_ZARR["1"]
x, y = 1000, 1000

# np.unique(FRAGMENTS_ZARR["1"]["surface_volume"][y:y+CROP_SIZE, x:x+CROP_SIZE, :])
fragment_cropped = fragment_id["surface_volume"][ x - CFG.BUFFER : x + CFG.BUFFER, y - CFG.BUFFER : y + CFG.BUFFER]
imageio.mimwrite("fragment_crop.mp4", (fragment_cropped.transpose(2, 0, 1)), "ffmpeg")
Video("fragment_crop.mp4", height=256, width=256)

In [ ]:
mask_cropped = fragment_id["truth"][y-CFG.BUFFER:y+CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER]
ir_cropped = fragment_id["infrared"][y-CFG.BUFFER:y+CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER]

plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plt.imshow(mask_cropped, cmap="gray")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(ir_cropped, cmap="gray")
plt.axis("off")
plt.show()

In [ ]:
# del fragment_id, fragment_cropped, mask_cropped, ir_cropped
# gc.collect()

In [ ]:
def extract_subvolume(location, volume):
    global printed
    y = location[0]
    x = location[1]
    subvolume = volume[y-CFG.BUFFER:y+CFG.BUFFER, x-CFG.BUFFER:x+CFG.BUFFER, :].astype(np.float32)
    
    return subvolume

In [ ]:
def visualize_dataset_patches(locations_ds, labels, mode: str, fold = 0):
    fig, ax = plt.subplots()
    ax.imshow(labels)

    for y, x in locations_ds:
        patch = patches.Rectangle([x - CFG.BUFFER, y - CFG.BUFFER], 2 * CFG.BUFFER, 2 * CFG.BUFFER, linewidth=2, edgecolor='g', facecolor='none')
        ax.add_patch(patch)
    plt.savefig(f"fold-{fold}-{mode}.png")
    plt.show()    

### Dataloaders

In [ ]:
import numpy as np
from albumentations.core.transforms_interface import ImageOnlyTransform

class MultichannelNoise(ImageOnlyTransform):

    def __init__(self, intensity=CFG.noise_intensity, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.intensity = intensity

    def apply(self, img, **params):
        intensity = np.random.uniform(*self.intensity)
        noise = np.random.normal(loc=0, scale=intensity*255, size=img.shape)
        img = img + noise
        return np.clip(img, 0, 255).astype(np.float32) # クリップして0から255の範囲に保つ


In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder

from albumentations.core.transforms_interface import ImageOnlyTransform

class SubvolumeDataset(Dataset):
    def __init__(self, locations, volume, labels, buffer, is_train: bool, return_location: bool = False):
        self.locations = locations
        self.volume = volume
        self.labels = labels        
        self.buffer = buffer
        self.is_train = is_train
        self.return_location = return_location

    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        label = None
        location = np.array(self.locations[idx])
        y, x = location[0], location[1]

        subvolume = extract_subvolume(location, self.volume)
        
        if self.labels is not None:
            label = self.labels[y - self.buffer:y + self.buffer, x - self.buffer:x + self.buffer]            
            label = np.stack([label], axis=-1)            
            
        # 段々meanは小さくなる
        mean = np.array([0.45 - i / 200 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        # 段々stdは小さくなる
        std = np.array([0.25 for i in range(0, CFG.Z_DIM)]).reshape(-1, 1, 1)
        
        if self.is_train and label is not None:          
            transformed = A.Compose([
                A.HorizontalFlip(p=0.2),
                A.VerticalFlip(p=0.2),                
                A.Transpose(p=0.2),
                # A.RandomScale(p=0.2, scale_limit=0.2),
                # A.RandomRotate90(p=0.2),                
                A.ShiftScaleRotate(p=0.4, scale_limit=0.2,),
                # A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
                # A.CoarseDropout(
                #     max_holes=1, 
                #     max_width=int(CFG.BUFFER * 2 * 0.3),
                #     max_height=int(CFG.BUFFER * 2 * 0.3), 
                #     mask_fill_value=0, 
                #     p=0.5
                # ),
                # A.GridDistortion(p=0.2),                
                # MultichannelNoise(
                #     p=0.1,
                # ),                
                A.PadIfNeeded(min_height=self.buffer * 2, min_width=self.buffer * 2),
                A.Resize(height=self.buffer * 2, width=self.buffer * 2),
            ])(image=subvolume, mask=label)
            subvolume = transformed["image"]
            label = transformed["mask"]
            subvolume = np.transpose(subvolume, (2, 0, 1))
            label = np.transpose(label, (2, 0, 1))
            subvolume /= 255.
            subvolume = (subvolume - mean) / std       
        else:
            if label is None:
                subvolume = np.transpose(subvolume, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
            else:
                # print("subvolume in val dataset (before aug)", subvolume, file=open("before-val-aug.log", "w")) 
                subvolume = np.transpose(subvolume, (2, 0, 1))
                label = np.transpose(label, (2, 0, 1))
                subvolume /= 255.
                subvolume = (subvolume - mean) / std
        # print("subvolume", subvolume)
        if self.return_location:
            return subvolume, location
        return subvolume, label        

### Model
* Encoder is a 3D ResNet model. The architecture has been modified to remove temporal downsampling between blocks.
* A 2D decoder is used for predicting the segmentation map.
* The encoder feature maps are average pooled over the Z dimension before passing it to the decoder.

In [ ]:
class SmpUnetDecoder(nn.Module):
	def __init__(
		self,
		in_channel,
		skip_channel,
		out_channel,
	):
		super().__init__()
		self.center = nn.Identity()

		i_channel = [
			in_channel,
		] + out_channel[:-1]
		s_channel = skip_channel
		o_channel = out_channel
		block = [
			DecoderBlock(i, s, o, use_batchnorm=True, attention_type=None)
			for i, s, o in zip(i_channel, s_channel, o_channel)
		]
		self.block = nn.ModuleList(block)

	def forward(self, feature, skip):
		d = self.center(feature)
		decode = []
		for i, block in enumerate(self.block):
			s = skip[i]
			d = block(d, s)
			decode.append(d)

		last = d
		return last, decode

In [ ]:
class Decoder(nn.Module):
    def __init__(self, encoder_dims, upscale):
        super().__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(encoder_dims[i]+encoder_dims[i-1], encoder_dims[i-1], 3, 1, 1, bias=False),
                nn.BatchNorm2d(encoder_dims[i-1]),
                nn.ReLU(inplace=True)
            ) for i in range(1, len(encoder_dims))])

        self.logit = nn.Conv2d(encoder_dims[0], 1, 1, 1, 0)
        self.up = nn.Upsample(scale_factor=upscale, mode="bicubic")

    def forward(self, feature_maps):
        for i in range(len(feature_maps)-1, 0, -1):
            f_up = F.interpolate(feature_maps[i], scale_factor=2, mode="bicubic")
            f = torch.cat([feature_maps[i-1], f_up], dim=1)
            f_down = self.convs[i-1](f)
            feature_maps[i-1] = f_down

        x = self.logit(feature_maps[0])
        # print("feature_map in decoder", list(map(lambda x: x.shape, feature_maps)))
        mask = self.up(x)
        return mask, feature_maps[0]

In [ ]:
class SegModel(nn.Module):
    def __init__(self,model_depth=CFG.MODEL_DEPTH):
        super().__init__()
        self.encoder = generate_model(model_depth=CFG.MODEL_DEPTH, n_input_channels=1)
        encoder_dims = [64, 128, 256, 512]
        self.decoder = Decoder(encoder_dims=encoder_dims, upscale=4)
  
        # self.logit1 = nn.Conv2d(encoder_dims[0], 1, 1, 1, 0)
        # self.up1 = nn.Upsample(scale_factor=4, mode="bicubic")
        
        self.encoder2_dim = [64, 128, 256, 512]  #
        self.decoder2_dim = [
            128,
            64,
            32,
        ]
        self.encoder2 = resnet10t(pretrained=CFG.pretrained, in_chans=64)

        self.decoder2 = SmpUnetDecoder(
            in_channel=self.encoder2_dim[-1],
            skip_channel=self.encoder2_dim[:-1][::-1],
            out_channel=self.decoder2_dim,
        )
        self.logit2 = nn.Conv2d(self.decoder2_dim[-1], 1, kernel_size=1)

    def forward(self, x: torch.Tensor):
        B, C, D, H, W = x.shape
        assert x.dim() == 5
        # print("initial x.shape", x.shape)
        feat_maps = self.encoder(x)
        feat_maps_pooled = [torch.mean(f, dim=2) for f in feat_maps]
        # print("feature map pooled", list(map(lambda a: a.shape, feat_maps_pooled)))
        logit1, last = self.decoder(feat_maps_pooled)
  
        # logit1 = self.logit1(last)
        # print("logit1.shape", logit1.shape)
        # logit1 = self.up1(logit1)
  
        x = last  # .detach()
        # x = F.avg_pool2d(x,kernel_size=2,stride=2)
        encoder = []
        e = self.encoder2
        x = e.layer1(x)
        encoder.append(x)
        x = e.layer2(x)
        encoder.append(x)
        x = e.layer3(x)
        encoder.append(x)
        x = e.layer4(x)
        encoder.append(x)

        feature = encoder[-1]
        skip = encoder[:-1][::-1]
        last, decoder = self.decoder2(feature, skip)
        logit2 = self.logit2(last)
        # print("logit2.shape", logit2.shape)
        logit2 = F.interpolate(
            logit2, size=(H, W), mode="bicubic", align_corners=False, antialias=True
        )
  
        return logit1, logit2
    
    def load_pretrained_weights(self, state_dict):
        # Convert 3 channel weights to single channel
        # ref - https://timm.fast.ai/models#Case-1:-When-the-number-of-input-channels-is-1
        conv1_weight = state_dict['conv1.weight']
        state_dict['conv1.weight'] = conv1_weight.sum(dim=1, keepdim=True)
        print(self.encoder.load_state_dict(state_dict, strict=False))

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder=SegModel()
        weight_path = f"r3d{CFG.MODEL_DEPTH}_K_200ep.pt"
        if CFG.pretrained:
            self.encoder.load_pretrained_weights(torch.load(weight_path)["state_dict"])

    def forward(self, images:torch.Tensor):
        #image.shape=(b,C,H,W)
        if images.ndim==4:
            images=images[:,None]
        # images=normalization(images)
        output = self.encoder(images)
        return output

In [ ]:
# class Decoder(nn.Module):
# 	def __init__(self, encoder_dims, upscale):
# 		super().__init__()
# 		self.convs = nn.ModuleList([
# 			nn.Sequential(
# 				nn.Conv2d(encoder_dims[i]+encoder_dims[i-1], encoder_dims[i-1], 3, 1, 1, bias=False),
# 				nn.BatchNorm2d(encoder_dims[i-1]),
# 				nn.ReLU(inplace=True)
# 			) for i in range(1, len(encoder_dims))])

# 		self.logit = nn.Conv2d(encoder_dims[0], 1, 1, 1, 0)
# 		self.up = nn.Upsample(scale_factor=upscale, mode="bicubic")

# 	def forward(self, feature_maps):
# 		for i in range(len(feature_maps)-1, 0, -1):
# 			f_up = F.interpolate(feature_maps[i], scale_factor=2, mode="bicubic")
# 			f = torch.cat([feature_maps[i-1], f_up], dim=1)
# 			f_down = self.convs[i](f)
# 			feature_maps[i-1] = f_down

# 		x = self.logit(feature_maps[0])
# 		mask = self.up(x)
# 		return mask, feature_maps[0]
	
# class SegModel(nn.Module):
# 	def __init__(self,model_depth=CFG.MODEL_DEPTH):
# 		super().__init__()
# 		self.encoder = generate_model(model_depth=CFG.MODEL_DEPTH, n_input_channels=1)
# 		encoder_dims = [64, 64, 128, 256, 512]
# 		self.decoder = Decoder(encoder_dims=encoder_dims, upscale=4)
  		
# 		self.weight1 = nn.ModuleList(
# 			[
# 				nn.Sequential(
# 					nn.Conv3d(dim, dim, kernel_size=3, padding=1),
# 	 				nn.BatchNorm3d(dim),
# 					nn.ReLU(inplace=True),
# 				)
# 				for dim in encoder_dims
# 			]
# 		)
  
# 		self.encoder2_dim = [64, 128, 256, 512]  #
# 		self.decoder2_dim = [
# 			128,
# 			64,
# 			32,
# 		]
# 		self.encoder2 = resnet10t(pretrained=CFG.pretrained, in_chans=64)

# 		self.decoder2 = SmpUnetDecoder(
# 			in_channel=self.encoder2_dim[-1],
# 			skip_channel=self.encoder2_dim[:-1][::-1],
# 			out_channel=self.decoder2_dim,
# 		)
# 		self.logit2 = nn.Conv2d(self.decoder2_dim[-1], 1, kernel_size=1)

		
# 	def forward(self, x: torch.Tensor):
# 		B, C, _, H, W = x.shape
# 		D = CFG.Z_DIM - CFG.BATCH_Z_DIFF
# 		assert x.dim() == 5
# 		# print(x.shape)
# 		xx = [
# 			x[:, :, i: i + CFG.Z_DIM - CFG.BATCH_Z_DIFF]
# 			for i in range(0, CFG.BATCH_Z_DIFF + 1, 2)
# 		]
# 		K = len(xx)
# 		x = torch.cat(xx, 0)
  
# 		# print("K:", K)
# 		# print("x.shape", x.shape)
		
# 		encoder = []
# 		e = self.encoder
# 		x = e.conv1(x)
# 		x = e.bn1(x)
# 		x = e.relu(x)		
# 		# print("first encoder: preprocess conv layer ok", x.shape)	
# 		encoder.append(x)
# 		x = e.maxpool(x)
# 		x = e.layer1(x)
# 		# print("first encoder: first layer ok", x.shape)	
# 		encoder.append(x)
# 		x = e.layer2(x)
# 		# print("first encoder: second layer ok", x.shape)	
# 		encoder.append(x)
# 		x = e.layer3(x)
# 		# print("first encoder: third layer ok", x.shape)	
# 		encoder.append(x)
# 		x = e.layer4(x)
# 		# print("first encoder: fourth layer ok", x.shape)	
# 		encoder.append(x)
# 		# print('encoder', [f.shape for f in encoder])

# 		for i in range(len(encoder)):
# 			e = encoder[i]
# 			f = self.weight1[i](e)
# 			_, _, _, h, w = e.shape
# 			f = rearrange(f, "(K B) c d h w -> B K c d h w", K=K, B=B, d=D, h=h, w=w)
# 			e = rearrange(e, "(K B) c d h w -> B K c d h w", K=K, B=B, d=D, h=h, w=w)
# 			w = F.softmax(f, 1)
# 			e = (w * e).sum(1)
# 			encoder[i] = e		
# 		# print("first encoder ok", list(map(lambda a: a.shape, encoder)))
# 		feat_maps_pooled = [torch.mean(f, dim=2) for f in encoder]
# 		# print("feature map pooled", list(map(lambda a: a.shape, feat_maps_pooled)))
# 		logit1, last = self.decoder(feat_maps_pooled[1:])
  
# 		# print("logit1.shape", logit1.shape)
# 		# print("last.shape", last.shape)
  
# 		# --- second Unet ---
# 		x = last  # .detach()
# 		# x = F.avg_pool2d(x,kernel_size=2,stride=2)
# 		encoder = []
# 		e = self.encoder2
# 		x = e.layer1(x)
# 		encoder.append(x)
# 		x = e.layer2(x)
# 		encoder.append(x)
# 		x = e.layer3(x)
# 		encoder.append(x)
# 		x = e.layer4(x)
# 		encoder.append(x)

# 		feature = encoder[-1]
# 		skip = encoder[:-1][::-1]
# 		last, decoder = self.decoder2(feature, skip)
# 		logit2 = self.logit2(last)
# 		logit2 = F.interpolate(
# 			logit2, size=(H, W), mode="bicubic", align_corners=False, antialias=True
# 		)

# 		# print("logit2.shape", logit2.shape)
  
# 		return logit1, logit2
	
# 	def load_pretrained_weights(self, state_dict):
# 		# Convert 3 channel weights to single channel
# 		# ref - https://timm.fast.ai/models#Case-1:-When-the-number-of-input-channels-is-1
# 		conv1_weight = state_dict['conv1.weight']
# 		state_dict['conv1.weight'] = conv1_weight.sum(dim=1, keepdim=True)
# 		print(self.encoder.load_state_dict(state_dict, strict=False))
		
   
# class CustomModel(nn.Module):
# 	def __init__(self):
# 		super().__init__()

# 		self.encoder=SegModel()
# 		weight_path = f"r3d{CFG.MODEL_DEPTH}_KM_200ep.pt"
# 		if CFG.pretrained:
# 			self.encoder.load_pretrained_weights(torch.load(weight_path)["state_dict"])

# 	def forward(self, images:torch.Tensor):
# 		#image.shape=(b,C,H,W)
# 		if images.ndim==4:
# 			images=images.unsqueeze(dim=1)
# 		# images=normalization(images)
# 		output = self.encoder(images)
# 		return output

In [ ]:
net_check_input = torch.from_numpy(np.random.randn(12, 12, 128, 128).astype(np.float32))
out = CustomModel()(net_check_input)

### Competition metric (F0.5 Score)

In [ ]:
# ref - https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
def fbeta_score(preds, targets, threshold, beta=0.5, smooth=1e-5):
    preds_t = torch.where(preds > threshold, 1.0, 0.0).float()
    y_true_count = targets.sum()
    
    ctp = preds_t[targets==1].sum()
    cfp = preds_t[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

### Training

In [ ]:
tc = torch
def TTA(x:tc.Tensor,model:nn.Module):
    #x.shape=(batch,c,h,w)
    shape=x.shape
    x=[x,*[tc.rot90(x,k=i,dims=(-2,-1)) for i in range(1,4)]]
    x=tc.cat(x,dim=0)
    _, x = model(x)
    x=x.reshape(4,shape[0], 1 ,*shape[2:])
    x=[tc.rot90(x[i],k=-i,dims=(-2,-1)) for i in range(4)]
    x=tc.stack(x,dim=0)
    return x.mean(0)

In [ ]:


class Model(pl.LightningModule):
    training_step_outputs = []
    validation_step_outputs = []
    test_step_outputs = [[], []]

    def __init__(self, **kwargs):
        super().__init__()

        self.model = CustomModel()        

        self.loss1 = CFG.loss1
        self.loss2 = CFG.loss2

    def forward(self, image, stage):
        if stage != "train":
            mask = TTA(image, self.model)
        else:
            mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        subvolumes, labels = batch

        image, labels = subvolumes.float(), labels.float()        
        assert image.ndim == 4
        
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0
        
        # print("labels", labels.max(), labels.min())

        assert labels.max() <= 1.0 and labels.min() >= 0

        if stage == "train":
            logit1, logit2 = self.forward(image, stage)
            loss = self.loss1(logit1, labels) * CFG.loss1_weight + CFG.loss2_weight * self.loss2(logit2, labels)
            logit = logit2
        else:
            logit = self.forward(image, stage)
            loss = self.loss1(logit, labels) * CFG.loss1_weight + CFG.loss2_weight * self.loss2(logit, labels)
        
        prob2 = torch.sigmoid(logit)

        pred_mask = (prob2 > CFG.threshold).float()
        
        # print("pred_mask", pred_mask)
        
        score = fbeta_score(pred_mask, labels, threshold=CFG.threshold)

        tp, fp, fn, tn = smp.metrics.get_stats(
            pred_mask.long(), labels.long(), mode="binary"
        )

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
            "score": score,
        }

    def shared_epoch_end(self, outputs, stage):
        # aggregate step metics
        tp = torch.cat([x["tp"] for x in outputs])
        fp = torch.cat([x["fp"] for x in outputs])
        fn = torch.cat([x["fn"] for x in outputs])
        tn = torch.cat([x["tn"] for x in outputs])
        loss = torch.mean(torch.Tensor([x["loss"] for x in outputs]))
        fbeta_score = torch.mean(torch.Tensor([x["score"] for x in outputs]))

        per_image_iou = smp.metrics.iou_score(
            tp, fp, fn, tn, reduction="micro-imagewise"
        )

        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro")

        metrics = {
            f"{stage}_per_image_iou": per_image_iou,
            f"{stage}_dataset_iou": dataset_iou,
            f"{stage}_loss": 10000 if loss.item() == 0 else loss.item(),
            f"{stage}_tp": tp.sum().int().item(),
            f"{stage}_fp": fp.sum().int().item(),
            f"{stage}_fn": fn.sum().int().item(),
            f"{stage}_tn": tn.sum().int().item(),
            f"{stage}_score": fbeta_score.item(),
        }

        self.log_dict(metrics, prog_bar=True, sync_dist=True)

    def training_step(self, batch, batch_idx):
        out = self.shared_step(batch, "train")
        self.training_step_outputs.append(out)
        return out

    def on_train_epoch_end(self):
        out = self.shared_epoch_end(self.training_step_outputs, "train")
        self.training_step_outputs.clear()
        return out

    def validation_step(self, batch, batch_idx):
        out = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(out)
        return out

    def on_validation_epoch_end(self):
        out = self.shared_epoch_end(self.validation_step_outputs, "valid")
        self.validation_step_outputs.clear()
        return out

    def test_step(self, batch, batch_idx):
        global predictions_map, predictions_map_counts

        patch_batch, loc_batch = batch

        loc_batch = loc_batch.long()
        patch_batch = patch_batch.float()
        predictions = self.forward(patch_batch, "test")
        predictions = predictions.sigmoid()
 
        predictions = torch.permute(predictions, (0, 2, 3, 1)).squeeze(dim=-1)
        predictions = (
            predictions.cpu().numpy()
        )
        loc_batch = loc_batch.cpu().numpy()
        
        self.test_step_outputs[0].extend(loc_batch)
        self.test_step_outputs[1].extend(predictions)        
        return loc_batch, predictions

    def on_test_epoch_end(self):
        global predictions_map, predictions_map_counts

        locs = np.array(self.test_step_outputs[0])
        preds = np.array(self.test_step_outputs[1])
        print("locs", locs.shape)
        print("preds", preds.shape)


        for (y, x), pred in zip(locs, preds):
            predictions_map[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += pred
            predictions_map_counts[
                y - CFG.BUFFER : y + CFG.BUFFER, x - CFG.BUFFER : x + CFG.BUFFER
            ] += 1
        
        predictions_map /= predictions_map_counts + CFG.exp

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=CFG.lr)

        scheduler = CFG.lr_scheduler(
            optimizer, T_0=CFG.num_epochs, T_mult=2, eta_min=CFG.eta_min_lr,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
        }

In [ ]:
sample_data_name = "2a"
sample_volume = FRAGMENTS_ZARR[sample_data_name]["surface_volume"]
sample_mask = FRAGMENTS_ZARR[sample_data_name]["mask"]
sample_label = FRAGMENTS_ZARR[sample_data_name]["truth"]
sample_locations = generate_locations_ds(sample_volume, sample_mask, sample_label, skip_zero=False)

In [ ]:
sample_ds = SubvolumeDataset(
    sample_locations,
    sample_volume,
    sample_label,
    CFG.BUFFER,
    is_train=True,
)

id = 240
sample_depth = CFG.Z_DIM // 2

fig, ax = plt.subplots(1, 2, figsize=(8, 6))

img = sample_ds[id][0][sample_depth, :, :]
label = sample_ds[id][1][0, :, :]
ax[0].imshow(img)
ax[1].imshow(label)

fig, ax = plt.subplots(figsize=(8, 6))

ax.imshow(sample_label)

y, x = sample_locations[id]
patch = patches.Rectangle([x - CFG.BUFFER, y - CFG.BUFFER], 2 * CFG.BUFFER, 2 * CFG.BUFFER, linewidth=2, edgecolor='g', facecolor='none')
ax.add_patch(patch)
plt.show()    

fig, ax = plt.subplots(CFG.Z_DIM, 1, figsize=(12, 24))

for i in range(CFG.Z_DIM):
    img, _ = sample_ds[id]
    img = img[i, :, :]
    ax[i].hist(img.flatten(), bins=1000)  # Plot histogram of the flattened data
    ax[i].set_title(f"Histogram of Channel {i}")  # Add title to the plot    
fig.tight_layout()
fig.show()

In [ ]:


k_folds = 4
kfold = KFold(
    n_splits=k_folds,
    shuffle=True
)

# 0: 1, 1: 2a（下側）, 2: 2b（上側）, 3: 3
data_list = [
    (
        FRAGMENTS_ZARR[key]["surface_volume"], 
        FRAGMENTS_ZARR[key]["truth"],
        FRAGMENTS_ZARR[key]["mask"],         
    ) for key in ["1", "2a", "2b", "3"]
]

predictions_map = None
predictions_map_counts = None

for fold, (train_data, val_data) in enumerate(kfold.split(data_list)):
    print(f'FOLD {fold}')
    print('--------------------------------')
    print("train_data", train_data)
    print("val_data", val_data)
    one = data_list[train_data[0]]
    two = data_list[train_data[1]]
    three = data_list[train_data[2]]
    train_volume = np.concatenate([one[0], two[0], three[0]], axis=1)
    train_label = np.concatenate([one[1], two[1], three[1]], axis=1)
    train_mask = np.concatenate([one[2], two[2], three[2]], axis=1)
    val_volume, val_label, val_mask = data_list[val_data[0]]    

    train_locations_ds = generate_locations_ds(train_volume, train_mask, train_label, skip_zero=True)
    val_location_ds = generate_locations_ds(val_volume, val_mask, skip_zero=False)

    visualize_dataset_patches(train_locations_ds, train_label, "train", fold)
    visualize_dataset_patches(val_location_ds, val_label, "val", fold)
    
    # Init the neural network
    model = Model()

    wandb.finish()
    # Initialize a trainer
    now = datetime.datetime.now()
    now = f"{now}".replace(" ", "")
    
    checkpoint_callback = pytorch_lightning.callbacks.ModelCheckpoint(
        monitor='valid_score',
        dirpath=os.path.join(CFG.DATA_DIR, f"best-{now}-fold-{fold}-val-{val_data[0]}"),
        mode="max",
        filename='3dcnn-train-fold-' + str(fold) + '-{epoch:02d}-{valid_score:.3f}',
        save_last=True,
    )
    
    trainer = pl.Trainer(
        max_epochs=CFG.num_epochs,
        precision=16,
        devices="0,1,2,3",
        accelerator="gpu",
        # strategy="ddp_find_unused_parameters_false",
        # strategy="ddp_fork",
        logger=WandbLogger(
            name=f"3dcnn-{now}-fold-{fold}-val-{val_data[0]}",
            notes=CFG.WANDB_NOTE,
            config=class2dict(CFG()),
        ),
        callbacks=[checkpoint_callback],
        default_root_dir=os.path.join(CFG.DATA_DIR, f"{now}-fold-{fold}-val-{val_data[0]}"),
    )
    
    # Sample elements randomly from a given list of ids, no replacement.
    train_ds = SubvolumeDataset(
        train_locations_ds,
        train_volume,
        train_label,
        CFG.BUFFER,
        is_train=True
    )
    val_ds = SubvolumeDataset(
        val_location_ds,
        val_volume,
        val_label,
        CFG.BUFFER,
        is_train=False,
    )

    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG.BATCH_SIZE,
        num_workers=CFG.num_workers,
        shuffle=True,
        # persistent_workers=True,
        # pin_memory=True,
    )
    val_loader = torch.utils.data.DataLoader(
        val_ds, 
        batch_size=CFG.BATCH_SIZE,
        num_workers=CFG.num_workers,
        shuffle=False,
        # pin_memory=True,
        # persistent_workers=True,
    )

    # Train the model
    trainer.fit(model, train_loader, val_loader)   
    
wandb.finish()
